# 🇪🇬 EgySentiment: Fine-Tune Llama 3 on Egyptian Financial News

**Author:** AI Research Scientist  
**Goal:** Fine-tune `Llama-3-8b-Instruct` to analyze sentiment (Positive/Negative/Neutral) of Egyptian financial news.  
**Environment:** Google Colab (Free Tier - T4 GPU)  

### 🚀 Workflow
1. **Setup:** Install Unsloth (optimized for low VRAM).
2. **Data:** Load & format `training_data.jsonl`.
3. **Model:** Load 4-bit Quantized Llama-3.
4. **Train:** SFT with LoRA adapters.
5. **Eval:** Confusion Matrix & Classification Report.
6. **Export:** Save as GGUF (`q4_k_m`) for local inference.

## 1. Setup & Installation

In [ ]:
%%capture
# Install Unsloth, Xformers (Flash Attention), and other deps
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install scikit-learn matplotlib seaborn

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4-bit pre-quantized models we support for 4x faster downloading + no OOMs.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

## 2. Data Loading & Formatting
We transform the raw JSONL into the Alpaca/Instruction format expected by the model.

In [ ]:
import json
from datasets import Dataset
import pandas as pd

# Load your dataset (upload training_data.jsonl to Colab files first)
dataset_path = "training_data.jsonl"

try:
    # Read JSONL
    data = []
    with open(dataset_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    
    df = pd.DataFrame(data)
    print(f"Loaded {len(df)} samples")
    
except FileNotFoundError:
    print("⚠️ Please upload 'training_data.jsonl' to the Colab runtime!")
    # Create dummy data for demonstration if file missing
    df = pd.DataFrame([
        {"text": "EGX30 rises 2%", "sentiment": "positive", "reasoning": "Market went up"},
        {"text": "CIB shares fall", "sentiment": "negative", "reasoning": "Shares dropped"}
    ] * 10)

# Define Prompt Template (Alpaca-style for Instruction Tuning)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Analyze the sentiment of the following Egyptian financial news article. Provide the sentiment (positive/negative/neutral) and a brief reasoning.

### Input:
{}

### Response:
{{"sentiment": "{}", "reasoning": "{}"}}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs       = examples["text"]
    sentiments   = examples["sentiment"]
    reasonings   = examples["reasoning"]
    texts = []
    for input, sentiment, reasoning in zip(inputs, sentiments, reasonings):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, sentiment, reasoning) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, "ground_truth_sentiment": sentiments }

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True)

# Split 90/10
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(f"Train size: {len(train_dataset)}")
print(f"Test size: {len(test_dataset)}")

## 3. Model Configuration (LoRA)
We add LoRA adapters to all linear layers for efficient fine-tuning.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## 4. Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set to 0 for full epochs, or use num_train_epochs
        # num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

## 5. Evaluation (Critical)
We run inference on the test set and calculate metrics.

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import re

y_true = []
y_pred = []

print("Running inference on test set...")

for i in range(len(test_dataset)):
    # Prepare input
    input_text = test_dataset[i]["text"].split("### Response:")[0] + "### Response:\n"
    ground_truth = test_dataset[i]["ground_truth_sentiment"]
    
    inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

    # Generate
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
    response = tokenizer.batch_decode(outputs)[0]
    
    # Extract JSON from response
    try:
        # Naive extraction: look for JSON-like structure after "Response:"
        generated_part = response.split("### Response:")[1]
        # Simple regex to find sentiment value
        match = re.search(r'"sentiment":\s*"(positive|negative|neutral)"', generated_part, re.IGNORECASE)
        if match:
            pred_sentiment = match.group(1).lower()
        else:
            pred_sentiment = "neutral" # Fallback
    except:
        pred_sentiment = "neutral"
        
    y_true.append(ground_truth)
    y_pred.append(pred_sentiment)

# Metrics
print("\nClassification Report:")
print(classification_report(y_true, y_pred, labels=["positive", "neutral", "negative"]))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred, labels=["positive", "neutral", "negative"])
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=["positive", "neutral", "negative"], 
            yticklabels=["positive", "neutral", "negative"])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## 6. Export to GGUF (Memory Safe)
We save the model to GGUF format for use in local LLM runners (LM Studio, Ollama).
**Note:** We set `maximum_memory_usage=0.6` to prevent RAM crashes on Colab.

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# Save to GGUF
if True: 
    model.save_pretrained_gguf(
        "model_gguf", 
        tokenizer, 
        quantization_method = "q4_k_m",
        maximum_memory_usage = 0.6, # CRITICAL for Colab Free Tier
    )
    print("✅ GGUF saved to 'model_gguf' directory!")

In [ ]:
# Download the GGUF file (Optional - if you want to download immediately)
# from google.colab import files
# files.download("model_gguf/unsloth.Q4_K_M.gguf")